<a href="https://colab.research.google.com/github/Nov05/flask-yelp-reviews/blob/master/notebooks/2019_09_07_yelp_reviews_flask_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-09-07
# https://github.com/Nov05/flask-yelp-reviews
# /notebooks

# Go to the app

https://yelp-reviews.herokuapp.com  

github  
https://github.com/Nov05/flask-yelp-reviews  

### reference
download spaCy model  
https://spacy.io/usage/models   

example target webpage   
https://www.yelp.com/biz/aunt-jakes-new-york    

How to make Python code concurrent with 3 lines  
https://dev.to/rhymes/how-to-make-python-code-concurrent-with-3-lines-of-code-2fpe

# Imports

In [0]:
!pip install flask_cors
# Successfully installed flask-cors-3.0.8

# py_scraper.py

In [40]:
%%writefile py_scraper.py
import pandas as pd
import requests
from lxml import html

def scraper(url_yelp, num_reviews=20):
    
    # get business id
    url_yelp = url_yelp.split('?')[0]
    biz = url_yelp.replace('https://www.yelp.com/biz/', '')
    # get url
    url_base = "https://www.yelp.com/biz/" 
    url_api = "/review_feed?sort_by=date_desc&start="
    url = url_base + biz + url_api + str(num_reviews * 5)

    colnames = ['date', 'text', 'rating']
    df_reviews = pd.DataFrame(columns=colnames)
    with requests.get(url, timeout=20) as response: 
        if response.status_code==200:
            string = response.json()
            div = html.fromstring(string['review_list'])
            dates = div.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
            texts = [e.text for e in div.xpath("//div[@class='review-content']/p")]
            ratings = div.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")
            df = pd.DataFrame([dates, texts, ratings]).T
            df.columns = colnames
            df['date'] = df['date'].apply(lambda x: x.strip())
            df_reviews = pd.concat([df_reviews, df], 
                                   ignore_index=True)
            del df
   
    return df_reviews

Overwriting py_scraper.py


In [0]:
# test
url_yelp = "https://www.yelp.com/biz/aunt-jakes-new-york"

url_base = "https://www.yelp.com/biz/" 
url_api = "/review_feed?sort_by=date_desc&start="
# get business id
if '?' in url_yelp: 
    biz = url_yelp.split('?')[0]
biz = url_yelp.replace(url_base, '')
url = url_base + biz + url_api + str(5*20)

print(biz)
print(url)

aunt-jakes-new-york
https://www.yelp.com/biz/aunt-jakes-new-york/review_feed?sort_by=date_desc&start=100


In [0]:
# test function scraper()
from py_scraper import scraper
url_yelp = "https://www.yelp.com/biz/aunt-jakes-new-york"
df_reviews = scraper(url_yelp)
print(df_reviews.shape)
df_reviews

(20, 3)


,date,text,rating
0,6/30/2019,Anthony the manger was incredible! I had a sur...,5.0 star rating
1,6/29/2019,Even though they moved next door still amazing...,5.0 star rating
2,6/29/2019,Enjoyed the best pasta dinner here with my wif...,5.0 star rating
3,6/29/2019,Meh...got gluten free and regular pasta. Food ...,3.0 star rating
4,6/29/2019,For sure one for the books. Was excited to try...,2.0 star rating
5,6/29/2019,"Nestled in Little Italy, this location is a bi...",3.0 star rating
6,6/29/2019,Omg omg omg... I LOVE Aunt Jake's!! The place...,5.0 star rating
7,6/29/2019,Aunt Jake's is ok. The meatball appetizer was ...,3.0 star rating
8,6/28/2019,We were there for an early dinner. Visited bas...,4.0 star rating
9,6/27/2019,"Went to this place with a group of friends, th...",4.0 star rating


### download spaCy language model

In this case, we didn't use the model (only the stop words).   
But I don't want to change the original code.   
Maybe I will use some spaCy function in the future.    

In [0]:
!wget https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz
!tar -xzvf en_core_web_sm-2.1.0.tar.gz

# Stop words
add customer stop words in this file

In [2]:
%%writefile stopwords.txt
friends
amazing
awesome
love
male
female
girl
boy
place
u
want
cuz
deaf
told
check
coffee
place
it 's
is
the
in that
to try
for
10
i had
the
had the
around the
at
at the
he told
he said
she said
i said
we said
you said
they said
is the
place for
1
2
3
4
5
6
7
8
9
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z

Overwriting stopwords.txt


In [7]:
# test stop words
with open('stopwords.txt', 'r') as f:
    str = f.read()
    set_stopwords = set(str.split('\n'))
print(set_stopwords)

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
print(STOP_WORDS)

nlp = spacy.load("en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0")
doc = nlp(' '.join(set_stopwords))
for i, token in enumerate(doc):
    print(token.text, token.is_stop)
    if i >= 20: break

{'awesome', '6', 'friends', 'place for', 'k', 'to try', '4', 'b', 'm', 'he told', 'girl', 'the', 'p', 'e', 'had the', 'u', 'd', 'coffee', 's', '10', 'is the', '8', 'place', "it 's", 'check', 'in that', 'i had', 'at the ', 'i', 'a', 'q', '2', '1', 'is', 'y', 'l', 'j', '9', 'c', '3', 'o', 'v', '5', 'around the', 'male', 'n', 'for', 'z', 't', 'at', 'h', '7', 'amazing', 'want', 'boy', 'told', 'f', 'r', 'cuz', 'w', 'g', 'deaf', 'female', 'x'}
{'both', 'whereas', 'he', 'ten', 'the', 'some', 'therefore', 'please', 'an', 'hereby', 'as', 'even', 'had', 'our', 'everything', 'myself', 'except', 'can', '‘m', 'do', 'off', 'mine', 'herself', 'might', 'nothing', 'a', 'among', 'get', 'through', 'with', 'further', 'quite', 'although', 'otherwise', 'herein', 'most', 'around the', 'hereupon', 'empty', 'other', 'somewhere', 'were', "'ll", "'s", 'eight', 'give', 'amazing', 'becoming', 'either', '’d', 'made', 'should', 'but', "'m", 'five', 'mostly', 'just', 'themselves', 'to try', 'say', 'nevertheless', 'wh

# py_processor.py

In [0]:
!pip install scattertext
# Successfully installed mock-3.0.5 scattertext-0.0.2.52

In [5]:
%%writefile py_processor.py
import pandas as pd
import spacy
import scattertext

def processor(df_reviews):
    
    if len(df_reviews)==0:
        return None
    
    nlp = spacy.load("en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0")

    # add stop words
    with open('stopwords.txt', 'r') as f:
        str = f.read()
        set_stopwords = set(str.split('\n'))
    nlp.Defaults.stop_words |= set_stopwords
 
    corpus = (scattertext.CorpusFromPandas(df_reviews, 
                                          category_col='rating', 
                                          text_col='text',
                                          nlp=nlp)
              .build()
              .remove_terms(nlp.Defaults.stop_words, ignore_absences=True)
             )

    term_freq_df = corpus.get_term_freq_df()
    term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')
    term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')

    df_high = term_freq_df.sort_values(by='highratingscore', 
                                       ascending = False)
    df_poor = term_freq_df.sort_values(by='poorratingscore', 
                                       ascending=False)

    df_high = df_high[['highratingscore', 'poorratingscore']]
    df_high['highratingscore'] = round(df_high['highratingscore'], 2)
    df_high['poorratingscore'] = round(df_high['poorratingscore'], 2)
    df_high = df_high.reset_index(drop=False)
    df_high = df_high.head(5)

    df_poor = df_poor[['highratingscore', 'poorratingscore']]
    df_poor['highratingscore'] = round(df_poor['highratingscore'], 2)
    df_poor['poorratingscore'] = round(df_poor['poorratingscore'], 2)
    df_poor = df_poor.reset_index(drop=False)
    df_poor = df_poor.head(5)

    df_terms = pd.concat([df_high, df_poor],
                         ignore_index=True)
    return df_terms

Overwriting py_processor.py


In [20]:
# test function processer()

from py_scraper import scraper
url_yelp1 = "https://www.yelp.com/biz/aunt-jakes-new-york"
url_yelp2 = "https://www.yelp.com/biz/densetsu-plano"
df_reviews = scraper(url_yelp2)

from py_processor import processor
df_terms = processor(df_reviews)
df_terms

,term,highratingscore,poorratingscore
0,sushi restaurant,1.00,0.03
1,sushi,0.96,0.00
2,delicious,0.95,0.00
3,tower,0.92,0.00
4,salmon,0.92,0.00
5,hostess,0.00,1.00
6,the hostess,0.00,1.00
7,rude,0.00,1.00
8,the same,0.00,1.00
9,table,0.00,1.00


### Scattertext

https://github.com/JasonKessler/scattertext  

Jason Kessler - Using Scattertext and the Python NLP Ecosystem for Text Visualization  
https://github.com/JasonKessler/scattertext     

Stop words  
https://github.com/JasonKessler/scattertext/issues/24  

In [29]:
import spacy
import scattertext
nlp = spacy.load("en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0")

# add custome stop words
with open('stopwords.txt', 'r') as f:
    str = f.read()
    set_stopwords = set(str.split('\n'))
nlp.Defaults.stop_words |= set_stopwords

corpus = (scattertext.CorpusFromPandas(df_reviews, 
                                      category_col='rating', 
                                      text_col='text',
                                      nlp=nlp)
          .build()
          .remove_terms(nlp.Defaults.stop_words, ignore_absences=True)
         )
# Here are the terms that differentiate the corpus from a general English corpus.
print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

['reggatone', 'hecc', 'densetsu', 'rudest', 'waitstaff', 'helpings', 'sushi', 'burrito', 'idk', 'disappoints']


In [30]:
## Here are the terms that are most associated with '5.0 star rating'
term_freq_df = corpus.get_term_freq_df()
term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')
print(list(term_freq_df.sort_values(by='highratingscore', ascending=False).index[:10]))

['sushi restaurant', 'sushi', 'delicious', 'tower', 'salmon', 'best sushi', 'places', 'favorite', 'burrito', 'my favorite']


In [23]:
## Here are the terms that are most associated with '1.0 star rating'
# term_freq_df = corpus.get_term_freq_df()
term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')
print(list(term_freq_df.sort_values(by='poorratingscore', ascending=False).index[:10]))

['hostess', 'the hostess', 'rude', 'the same', 'table', 'did not', 'there is', 'should be', 'decided to', 'the tables']


In [32]:
corpus = (scattertext.CorpusFromPandas(df_reviews, 
                                      category_col='rating', 
                                      text_col='text',
                                      nlp=nlp)
          .build()
#           .remove_terms(nlp.Defaults.stop_words, ignore_absences=True)
         )
html = scattertext.produce_scattertext_explorer(corpus,
          category='5.0 star rating',
          category_name='5.0 star rating',
          not_category_name='Other rating',
          width_in_pixels=500,
#           metadata=df_reviews['date'],
       )
filename = "Review-Visualization.html"
open(filename, 'wb').write(html.encode('utf-8'))
import IPython
IPython.display.HTML(filename=filename)

# Flask app

example  
https://colab.research.google.com/drive/1d7uE8BwwSO3MTfP4IYoV0e12Qj2v-6jv?authuser=1#scrollTo=Yqks6lkzt-Fb   

another example   
https://realpython.com/flask-by-example-part-3-text-processing-with-requests-beautifulsoup-nltk/  

### for the flask app

In [0]:
!mkdir templates

In [37]:
%%writefile templates/index.html
<html>
    <h1>
        Welcome to the "Yelp Review NLP" app!
    </h1>
    <p><b>Call API at</b></p>
    <p>https://yelp-reviews.herokuapp.com/api/terms</p>
    <p><b>Request JSON example</b></p>
    <p>{"yelp_url": "https://www.yelp.com/biz/galli-new-york-5"}</p>
</html>

Overwriting templates/index.html


## app.py

In [20]:
%%writefile app.py
from flask import Flask, render_template, \
                  request, jsonify
from flask_cors import CORS

from py_scraper import scraper
from py_processor import processor

app=Flask(__name__)
CORS(app)

@app.route('/') 
@app.route('/index')
def index():
    return render_template('index.html')

############################################################
# API - terms
############################################################
@app.route('/api/terms', methods = ['POST'])
def api_terms():
    if request.method == 'POST':
        data_in = request.get_json(force=True)
        # e.g. "https://www.yelp.com/biz/aunt-jakes-new-york"
        url_yelp = data_in['yelp_url']
        df_reviews = scraper(url_yelp)
        df_terms = processor(df_reviews)
        data_out = df_terms.to_json(orient='index')

        return data_out

if __name__ == '__main__':
    app.run(debug=True)

Overwriting app.py


In [7]:
import pandas as pd
# df_terms = pd.DataFrame()
# data_out = df_terms.to_json(orient='index')
df_terms = None
data_out

'{}'

# Test API

### Test Heroku app  
https://yelp-reviews.herokuapp.com/  

In [33]:
import json
input1 = json.dumps({"yelp_url": "https://www.yelp.com/biz/aunt-jakes-new-york"})
input2 = json.dumps({"yelp_url": "https://www.yelp.com/biz/densetsu-plano?osq=densetsu"})
input3 = json.dumps({"yelp_url": "https://www.yelp.com/biz/galli-new-york-5"})

import requests
url = 'https://yelp-reviews.herokuapp.com/api/terms'

response = requests.post(url, input3)
print(response.status_code)
if response.status_code==200:
    print(response.text)
    print(json.dumps(json.loads(response.text), 
                     indent=4, sort_keys=True))

200
{"0":{"term":"drink","highratingscore":1.0,"poorratingscore":0.0},"1":{"term":"soho","highratingscore":1.0,"poorratingscore":0.0},"2":{"term":"friendly","highratingscore":1.0,"poorratingscore":0.0},"3":{"term":"the bartender","highratingscore":1.0,"poorratingscore":0.02},"4":{"term":"server","highratingscore":1.0,"poorratingscore":0.02},"5":{"term":"bag","highratingscore":0.01,"poorratingscore":1.0},"6":{"term":"rude","highratingscore":0.1,"poorratingscore":1.0},"7":{"term":"she could","highratingscore":0.1,"poorratingscore":1.0},"8":{"term":"terrible","highratingscore":0.1,"poorratingscore":1.0},"9":{"term":"hostesses","highratingscore":0.1,"poorratingscore":1.0}}
{
    "0": {
        "highratingscore": 1.0,
        "poorratingscore": 0.0,
        "term": "drink"
    },
    "1": {
        "highratingscore": 1.0,
        "poorratingscore": 0.0,
        "term": "soho"
    },
    "2": {
        "highratingscore": 1.0,
        "poorratingscore": 0.0,
        "term": "friendly"
    },


### Test locally  
Testing notebook  
https://colab.research.google.com/drive/1wyJoBblEKeOB3i9aKVHgJuEmlfV_Xqtm?authuser=1#scrollTo=ukfei479cSqn

In [0]:
############################################################
# you can ignore this code block
############################################################
!python app.py
# why does this cause error
# is it because that for the scraping function 
# the port has to be tunneled out?
'''
MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=5000): 
Max retries exceeded with url: /api/terms
(Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdd302b70b8>: 
Failed to establish a new connection: [Errno 111] Connection refused',))
'''

In [0]:
############################################################
# Install ngrok (do it once, for testing in Colab only)
############################################################
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [8]:
# run this code block to get a link, then run your Dash/Flask app
get_ipython().system_raw('./ngrok http 5000 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

!python app.py # this would run in Python 3.6
# !FLASK_APP=app.py flask run #【Issue】why would it run in Python 2.7?
# !FLASK_APP=app.py FLASK_ENV=development flask run

# for this app, test the api in another Colab notebook with
# https://9ea7ec2e.ngrok.io/api/terms (ngrok url chanages every time)

http://c885cc44.ngrok.io
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 329-983-874
127.0.0.1 - - [09/Sep/2019 02:18:39] "POST /api/terms HTTP/1.1" 200 -


# JSON

In [0]:
json_input = json.dumps({"yelp_url": "https://www.yelp.com/biz/aunt-jakes-new-york"})
json_input

In [0]:
import json
json_output = df_terms.to_json(orient='index')
print(json.dumps(json.loads(json_output), indent=4, sort_keys=True))

# Original app.py

There is no need to use `request.Session()` and `concurrent.futures.ThreadPoolExecutor` in this case. The web scraping will only take one url at a time.  

In [0]:
import numpy as np
import pandas as pd
from itertools import count

import spacy
from spacy.lang.en import English
from concurrent.futures import ThreadPoolExecutor as Executor

import json
from json import loads
import scattertext as st

import pickle
from flask_cors import CORS
from flask import Flask, render_template, \
                  request, jsonify
import os
import warnings
import wget
import tarfile
import requests
from requests import Session

path = "./down_sm/en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0"
nlp = sapcy.load(path)

def ValuePredictor(url_yelp, from_isbn=False):
    '''Takes a url, scrape site for reviews
    and calculates the term frequencies 
    sorts and returns the top 10 as a json object
    containing term, highratingscore, poorratingscore.'''

    # add business id
    url_base = "https://www.yelp.com/biz/" 
    url_api = "/review_feed?sort_by=date_desc&start="
    bid = url_yelp.replace('https://www.yelp.com/biz/', '')
    # deletes everything after "?" in url
    if '?' in url_yelp: 
        bid = url_yelp.split('?')[0]

    class Scraper():
        def __init__(self):
            self.data = pd.DataFrame()

        def get_data(self, n, bid=bid):
            # makes an http get request to given url 
            # and returns response as json
            
            with Session() as s:
                # converts json response into a dictionary
                
                with s.get(base_url+bid+api_url+str(n*20)) as resp: 
                    r = dict(resp.json()) 
                    _html = html.fromstring(r['review_list']) #loads from dictionary

                    dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                    reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                    ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                    df = pd.DataFrame({'dates': dates, 
                                       'reviews': reviews, 
                                       'ratings': ratings})

                    self.data = pd.concat([self.data,df])
        
        def scrape(self): 
            # make it faster
            # multithreaded looping
            
            with Executor(max_workers=40) as e:
                list(e.map(self.get_data, range(10)))

    s = Scraper()
    s.scrape()
    df = s.data

    nlp.Defaults.stop_words |= {'will', 'not', 'friends',
                                'amazing', 'awesome', 'first',
                                'he', 'check-in', '=', '= =',
                                'male','u','want', 'u want',
                                'cuz','him',"i've", 'deaf',
                                'on', 'her','told','told him',
                                'ins', 'check-ins','check-in',
                                'check','I', 'i"m', 'i', ' ',
                                'it', "it's", 'it.','they',
                                'coffee','place','they', 'the', 
                                'this','its', 'l','-','they',
                                'this','don"t','the ', ' the', 
                                'it', 'i"ve', 'i"m', '!', 
                                '1','2','3','4', '5','6','7',
                                '8','9','0','/','.',','}

    corpus = st.CorpusFromPandas(df, 
                                 category_col=2, 
                                 text_col=1,
                                 nlp=nlp).build()

    term_freq_df = corpus.get_term_freq_df()
    term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')
    term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')
 
    dp = term_freq_df.sort_values(by= 'poorratingscore', ascending = False)
    for i in dp.index: 
        if ' ' in i:
            dp = dp.drop([i])
            
    df = term_freq_df.sort_values(by= 'highratingscore', ascending = False)
    for i in df.index: 
        if ' ' in i:
            df = df.drop([i])
            
    df = df[['highratingscore', 'poorratingscore']]
    df['highratingscore'] = round(df['highratingscore'], 2)
    df['poorratingscore'] = round(df['poorratingscore'], 2)
    df = df.reset_index(drop=False)
    df = df.head(5)

    dp = dp[['highratingscore', 'poorratingscore']]
    dp['highratingscore'] = round(dp['highratingscore'], 2)
    dp['poorratingscore'] = round(dp['poorratingscore'], 2)
    dp = dp.reset_index(drop=False)
    dp = dp.head(5)
    df = pd.concat([df,dp])
    
    return df.to_dict('records')

# app
app=Flask(__name__)
CORS(app)

# routes
@app.route('/')#defaults to this just in case, legacy reasons
@app.route('/index')
def index():
    return flask.render_template('index.html')#we are going to have a form

# hold and run the results page
@app.route('/result', methods = ['POST'])
def result(): 
    # will capture our predictions, handles result
    
#     content_type = request.headers["content-type"]
#     if content_type == "application/json":
    if request.method == 'POST':
        to_predict_list = request.values['yelp_url']
        result = ValuePredictor(to_predict_list)
    #         return jsonify(test)

        response = app.response_class(
            response=json.dumps(result),
            status=200,
            mimetype='application/json')

        return response
#     else:
#         return jsonify("Content-Type is not application/json")

# app run
if __name__ == '__main__':
    app.run(port=9000, debug=True)

# Virtualenv
install packages, generate requirements.txt

In [0]:
!pip install pipenv
# Installing collected packages: virtualenv-clone, virtualenv, pipenv
# Successfully installed pipenv-2018.11.26 virtualenv-16.7.5 virtualenv-clone-0.5.3

In [0]:
## remove virtualenv
# !rm -rf /root/.local/share/virtualenvs/content-cQIIIOO2
# pipenv --rm

In [2]:
# !pipenv install ipykernel
!pipenv shell

Creating a virtualenv for this project…
Pipfile: /content/Pipfile
Using /usr/bin/python3 (3.6.8) to create virtualenv…
⠧ Creating virtual environment...Already using interpreter /usr/bin/python3
Using base prefix '/usr'
New python executable in /root/.local/share/virtualenvs/content-cQIIIOO2/bin/python3
Also creating executable in /root/.local/share/virtualenvs/content-cQIIIOO2/bin/python
Installing setuptools, pip, wheel...
done.

✔ Successfully created virtual environment! 
Virtualenv location: /root/.local/share/virtualenvs/content-cQIIIOO2
Creating a Pipfile for this project…
Launching subshell in virtual environment…
in/activate
ext==0.0.2.52 flask flask_cors gunicorn
Installing numpy…
Adding numpy to Pipfile's [packages]…
✔ Installation Succeeded 
Installing pandas…
Adding pandas to Pipfile's [packages]…
✔ Installation Succeeded 
Installing spacy…
Adding spacy to Pipfile's [packages]…
✔ Installation Succeeded 
Installing scattertext==0.0.2.52…
Adding scattertext to Pipfile's [pac

In [0]:
pipenv install numpy pandas lxml spacy scattertext==0.0.2.52 flask flask_cors gunicorn
pipenv run pip freeze > requirements.txt
pipenv lock

### (content) root@595beb394144:/content#
# python -m ipykernel install --user --name=content
# jupyter notebook
""" Why is python2.7 get called here? I need to fix this.
/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: 
UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
"""

# Generate other files

example app repo   
https://github.com/Nov05/DS-Unit-3-Sprint-4-Build-Week-Safe-Routes  

In [80]:
%%writefile Procfile
web: gunicorn app:app

Writing Procfile


In [14]:
%%writefile .slugignore
README.md 
__pycache__
datasets/  
notebooks/
images/
.ipynb_checkpoints/

Writing .slugignore


In [15]:
%%writefile .gitignore
__pycache__
.ipynb_checkpoints/

Writing .gitignore


In [85]:
!ls

app.py		      Pipfile.lock  py_processor.py   stopwords.txt
en_core_web_sm-2.1.0  Procfile	    py_scraper.py     templates
Pipfile		      __pycache__   requirements.txt


# Deploy app on Heroku   
 
simple tutorial   
https://github.com/Nov05/Lambda-School-Data-Science/blob/master/miscellaneous/2019-08-23%20Heroku%20CLI%20deployment%20terminal%20logs.md  

1. go to heroku.com
2. create an app
3. on the `deploy` page you will see CLI instructions

In [0]:
# Install CLI in Linux (Google Colab)
!curl https://cli-assets.heroku.com/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1892  100  1892    0     0   7629      0 --:--:-- --:--:-- --:--:--  7659
Installing CLI from https://cli-assets.heroku.com/heroku-linux-x64.tar.xz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.0M  100 17.0M    0     0  5610k      0  0:00:03  0:00:03 --:--:-- 5610k
v11.14.0
heroku installed to /usr/local/bin/heroku
heroku/7.29.0 linux-x64 node-v11.14.0


# Push files to github   
https://medium.com/@navan0/how-to-push-files-into-github-from-google-colab-379fd0077aa8   


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!rm -rf /content/.git

In [0]:
!sh /content/drive/My\ Drive/config/github-flask-yelp-reviews.sh

In [0]:
!git pull origin master

In [0]:
!git add .
!git commit -m "2019-09-08 first commit from google colab"

In [0]:
!git push --set-upstream origin master

In [3]:
!git push

Everything up-to-date


# Download files

In [0]:
!rm -rf template
!rm -rf sample_data
!rm -rf ngrok-stable-linux-amd64.zip
!rm -rf /content/en_core_web_sm-2.1.0.tar.gz
!rm -rf /content/Review-Visualization.html
!rm -rf /content/ngrok

In [18]:
# download folder. if failed, do it again.
%cd /content
# !zip -r colab.zip app.py Pipfile.lock py_scraper.py	templates Procfile README.md en_core_web_sm-2.1.0 requirements.txt Pipfile py_processor.py stopwords.txt
!zip -r colab.zip /content
from google.colab import files
files.download("colab.zip")

/content
updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/configurations/ (stored 0%)
updating: content/.config/configurations/config_default (stored 0%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2019.08.27/ (stored 0%)
updating: content/.config/logs/2019.08.27/16.17.17.407104.log (deflated 85%)
updating: content/.config/logs/2019.08.27/16.17.07.367707.log (deflated 54%)
updating: content/.config/logs/2019.08.27/16.17.00.530950.log (deflated 90%)
updating: content/.config/logs/2019.08.27/16.17.22.686467.log (deflated 55%)
updating: content/.config/logs/2019.08.27/16.17.21.663235.log (deflated 54%)
updating: content/.config/active_config (stored 0%)
updating: content/.config/.last_update_check.json (deflated 22%)
updating: content/.config/gce (stored 0%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/.metricsUUID (stored 0%)
updating: content/Pipfile.lock (deflated 60%)
updat